In [1]:
%load_ext autoreload
%autoreload 2

import json
import os.path as osp
import os,sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

from torch_geometric.utils import to_undirected
from torch_cluster import radius_graph, knn_graph
from torch_geometric.datasets import MNISTSuperpixels
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm
import argparse
import warnings
warnings.simplefilter('ignore')
from time import strftime, gmtime

sys.path.append('../')
import utils
import model.net as net
import model.data_loader as data_loader
from evaluate import evaluate

In [2]:
torch.__version__

'1.10.2+cu102'

In [3]:
!python --version

Python 3.8.8


### Import data

In [4]:
train_dl = torch.load('train_dataloader.pth')
test_dl = torch.load('test_dataloader.pth')

In [5]:
test_data = None
for cnt, test_data in enumerate(test_dl):
    if cnt == 5:
        break
    # data_module = data
    # x_cont = data.x[:, :6]  # include puppi
    # x_cat = data.x[:, 2:].long()
    # cnt += 1

### Send data to GPU

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
x_cont_test = test_data.x[:128, :6].to(device)  # include puppi
x_cat_test = test_data.x[:128, 6:].long().to(device)
etaphi_test = torch.cat([test_data.x[:128, 3][:, None], test_data.x[:128, 4][:, None]], dim=1).to(device=device)
batch = test_data.batch[:128].to(device)
edge_index_test = radius_graph(etaphi_test, r=0.4, batch=batch, loop=False, max_num_neighbors=255).to(device=device)
norm = torch.tensor([1., 1., 1., 1., 1., 1.]).to(device=device)

In [8]:
print(f'x_cont_test: {x_cont_test.shape}')
print(f'x_cat_test: {x_cat_test.shape}')
print(f'etaphi: {etaphi_test.shape}')
print(f'batch: {batch.shape}')
print(f'edge_index: {edge_index_test.shape}')

x_cont_test: torch.Size([128, 6])
x_cat_test: torch.Size([128, 2])
etaphi: torch.Size([128, 2])
batch: torch.Size([128])
edge_index: torch.Size([2, 396])


### Load Torch Script Jit Model

In [9]:
prefix = '../ckpts_April18_ReLU_responseCorr_momentumscaled128/'

In [10]:
# model_jit = torch.jit.load(f'{prefix}/MODELS/scripted_model_epoch1.pt').to(device)

In [11]:
# print("Parameters of the jit model:")
# for name, param in model_jit.named_parameters():
#     print(f"{name}: {param.size()}")
#     print(param)

In [12]:
# results_jit = model_jit(x_cont_test, x_cat_test, edge_index_test, batch)
# results_jit

### Restore ckpts

In [13]:
restore_ckpt = osp.join(prefix, 'last.pth.tar')

In [14]:
model_new = net.Net(continuous_dim=6, categorical_dim=2 , norm=norm).to(device)

In [15]:
param_restored_new = utils.load_checkpoint(restore_ckpt, model_new)
param_restored_new

{'epoch': 99,
 'state_dict': OrderedDict([('graphnet.embed_charge.weight',
               tensor([[ 0.5420, -1.0757, -0.9354,  0.1327, -1.1908,  0.0210, -0.4464,  0.0942],
                       [ 0.9703, -0.8516, -0.2811, -0.6934, -0.1244,  0.5209,  0.9734, -0.5901],
                       [-1.0361, -0.7385,  0.6947,  0.2093, -0.1846, -1.0023,  0.5991,  0.2141]],
                      device='cuda:0')),
              ('graphnet.embed_pdgid.weight',
               tensor([[-1.7275e+00, -3.1177e-01, -2.0489e-01,  7.6640e-01, -4.0755e-01,
                         1.9370e-01, -4.0166e-01,  7.8586e-01],
                       [-3.3877e-01,  1.0639e+00,  7.0922e-01,  8.5192e-01, -1.2576e-01,
                         8.8327e-01, -2.4952e-01, -1.4938e-01],
                       [ 5.3967e-01, -1.7436e+00, -6.8639e-02,  2.9134e-02, -7.9007e-01,
                        -7.4452e-01,  7.4446e-02, -1.0726e+00],
                       [-1.3122e-01,  5.0800e-01, -4.4229e-01,  1.6049e+00, -1.1198e-01

In [ ]:
results_new = model_new(x_cont_test, x_cat_test, edge_index_test, batch)
results_new/128

In [17]:
# Print parameters of the instantiated model
print("Parameters of the instantiated model:")
for name, param in model_new.named_parameters():
    print(f"{name}: {param.size()}")
    print(param)

Parameters of the instantiated model:
graphnet.embed_charge.weight: torch.Size([3, 8])
Parameter containing:
tensor([[ 0.5420, -1.0757, -0.9354,  0.1327, -1.1908,  0.0210, -0.4464,  0.0942],
        [ 0.9703, -0.8516, -0.2811, -0.6934, -0.1244,  0.5209,  0.9734, -0.5901],
        [-1.0361, -0.7385,  0.6947,  0.2093, -0.1846, -1.0023,  0.5991,  0.2141]],
       device='cuda:0', requires_grad=True)
graphnet.embed_pdgid.weight: torch.Size([7, 8])
Parameter containing:
tensor([[-1.7275e+00, -3.1177e-01, -2.0489e-01,  7.6640e-01, -4.0755e-01,
          1.9370e-01, -4.0166e-01,  7.8586e-01],
        [-3.3877e-01,  1.0639e+00,  7.0922e-01,  8.5192e-01, -1.2576e-01,
          8.8327e-01, -2.4952e-01, -1.4938e-01],
        [ 5.3967e-01, -1.7436e+00, -6.8639e-02,  2.9134e-02, -7.9007e-01,
         -7.4452e-01,  7.4446e-02, -1.0726e+00],
        [-1.3122e-01,  5.0800e-01, -4.4229e-01,  1.6049e+00, -1.1198e-01,
         -1.6305e+00,  5.1570e-01,  4.7604e-01],
        [-7.5163e-01, -9.1461e-01, -6.

In [19]:
#import numpy as np
#import matplotlib.pyplot as plt

#ratio = results_new.cpu().detach().numpy() / results_jit.cpu().detach().numpy()
#plt.plot(range(len(ratio)), ratio)